In [3]:
import sys
sys.path.append('../code')

In [4]:
import json
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [5]:
from make_patch_df import *
from patch_nlp import get_texts_and_labels

In [3]:
hero_name_set = get_hero_names()
ability_set = get_abi_names('../../dota2-database/json/abilities.json')

In [ ]:
patch_url_dict = {}
patch_url_dict['6.86'] = 'December_16,_2015_Patch'
patch_url_dict['6.85'] = 'September_24,_2015_Patch'
patch_url_dict['6.84'] = 'April_30,_2015_Patch'
patch_url_dict['6.83'] = 'December_17,_2014_Patch'
patch_url_dict['6.82'] = 'September_25,_2014_Patch'
patch_url_dict['6.81'] = 'April_29,_2014_Patch'
patch_url_dict['6.80'] = 'January_29,_2014_Patch'
patch_url_dict['6.79'] = 'October_21,_2013_Patch'
patch_url_dict['6.78'] = 'June_04,_2013_Patch'
patch_url_dict['6.77'] = 'December_19,_2012_Patch'

In [17]:
with open('../data/patch_685_df.pkl') as f:
    df_685_full = pickle.load(f)
    
# df_685 = df_from_patch_query('September_24,_2015_Patch', hero_name_set, '6.85')

In [19]:
df_685_full.head(6)

,hero,patch,text,num_buffs,num_nerfs,change_type,text_no_abi
0,Abaddon,6.85,"[Mist Coil cooldown reduced from 5 to 4.5, Bor...",2,0,"[1, 1]","[cooldown reduced from 5 to 4.5, Aghanim's Sce..."
1,Alchemist,6.85,[Acid Spray damage increased from 12/16/20/24 ...,2,0,"[1, 1]",[damage increased from 12/16/20/24 to 15/20/25...
2,Ancient Apparition,6.85,[Cold Feet cast range increased from 700 to 70...,1,0,[1],[cast range increased from 700 to 700/800/900/...
3,Anti-Mage,6.85,[Blink distance rescaled from 1000/1075/1150/1...,0,2,"[-1, -1]",[distance rescaled from 1000/1075/1150/1150 to...
4,Arc Warden,6.85,[Spark Wraith cooldown reduced from 7 to 4],1,0,[1],[Spark Wraith cooldown reduced from 7 to 4]
5,Axe,6.85,[],0,0,[],[]


In [ ]:
texts_685, labels_685 = get_texts_and_labels(df_685_full, labeled=True)

In [7]:
df_684 = df_from_patch_query('April_30,_2015_Patch', hero_name_set, '6.84')
df_684.head(4)
df_684['text_no_abi'] = df_684['text'].apply(remove_abi_names, abi_set=ability_set)
texts_684, labels_684 = get_texts_and_labels(df_684, labeled=False)

In [53]:
tfidf_train = TfidfVectorizer(use_idf=False)
tfmat_train_685 = tfidf_train.fit_transform(texts_685)
tfmat_test_684 = tfidf_train.transform(texts_684)

gdb_685 = GradientBoostingClassifier(random_state=0)
gdb_685.fit(tfmat_train_685.toarray(), labels_685)
gdb_685.predict_proba(tfmat_test_684.toarray())[:,1][:10]

array([ 0.89363513,  0.819224  ,  0.51470196,  0.93845754,  0.63061521,
        0.89363513,  0.77128835,  0.77128835,  0.85539399,  0.26799292])

In [50]:
model_output = np.round(gdb_685.predict_proba(tfmat_test_684.toarray())[:,1], 4)
counter = 0
predictions = []
for item in df_684['text']:
    predictions.append(model_output[counter:counter+len(item)])
    counter += len(item)
df_684['predictions'] = predictions

In [58]:
df_684.text[6]

[u'Brain Sap mana cost from 125/150/175/200 to 100/125/150/175',
 u'Units affected by  Nightmare no longer provide vision',
 u'Nightmare End can now be used by Bane to end all other existing Nightmares']

In [59]:
df_684.text[1]

[u'Unstable Concoction:\n\n Countdown can now be seen by enemies\n\n Now has an area targeting cursor',
 u"Greevil's Greed now grants 4x gold bounty from Runes",
 u"Alchemist can now cast  Aghanim's Scepter to directly grant any allied hero all Aghanim's Scepter bonuses as a buff (the hero upgrade and the stat upgrade). The scepter is consumed in the process. Multiple instances of this buff do not stack. Alchemist can target himself as well"]